In [0]:
import pandas  as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
train = pd.read_csv('train.csv')

In [0]:
train.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [0]:
train.columns

Index(['id', 'original_text', 'lang', 'retweet_count', 'original_author',
       'sentiment_class'],
      dtype='object')

In [0]:
train.shape

(3235, 6)

In [0]:
train_clean = []
for i in range(0, 3235):
  review = re.sub('[^a-zA-Z]',' ',train['original_text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review =' '.join(review)
  train_clean.append(review)

In [0]:
train_clean[0:10]

['happi mothersday amaz mother know hard abl see mother today us protect vulner member societi beatcoronaviru pic twitter com va nfjfq b',
 'happi mother day mum sorri bring mother day flower cwtch honestli point walk hot coal abl bell soon love lot xxx p need photo http photo app goo gl vxblrsczd te',
 'happi mother day mother day work today quiet time reflect dog walk finish jigsaw garden learn guitar chord drunk strawberri gin tonic watch lee even dvd favourit place visit isol pic twitter com gz xvvf f',
 'happi mother day beauti woman royalti sooth mummi jeremi emerald prayforroksi ultimateloveng pic twitter com oeeti pvv',
 'rememb amaz ladi made late grandmoth iri mum carol great grandmoth ethel miss never forgotten happi mother day great mum love sent xxxx pic twitter com xzzzdeybj',
 'happi mother day bratz world familiez yasmin mum portia bratz bratzworldfamiliez bratz bratzkidz bratzyasmin bratzportia theonlygirlswithapassionforfashion bratzdol bratzcollector happymothersday 

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=7319)
X= cv.fit_transform(train_clean).toarray()
y= train.iloc[: , 5].values
y

array([ 0,  0, -1, ...,  0,  0, -1])

In [0]:
X.shape

(3235, 7319)

In [0]:
test = pd.read_csv('test.csv')

In [0]:
test.head()

,id,original_text,lang,retweet_count,original_author
0,1.246628e+18,"3. Yeah, I once cooked potatoes when I was 3 y...",en,0,LToddWood
1,1.245898e+18,"Happy Mother's Day to all the mums, step-mums,...",en,0,iiarushii
2,1.244717e+18,"I love the people from the UK, however, when I...",en,0,andreaanderegg
3,1.245730e+18,Happy 81st Birthday Happy Mother’s Day to my m...,en,1,TheBookTweeters
4,1.244636e+18,Happy Mothers day to all those wonderful mothe...,en,0,andreaanderegg


In [0]:
test.shape

(1387, 5)

In [0]:
test_clean = []
for i in range(0, 1387):
  review = re.sub('[^a-zA-Z]',' ',test['original_text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review =' '.join(review)
  test_clean.append(review)

In [0]:
print(len(test_clean))

1387


In [0]:
test_cv = CountVectorizer()
test_count= cv.fit_transform(test_clean).toarray()
test_count

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
test_count.shape

(1387, 7319)

In [0]:
#spliting the dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
train.columns

Index(['id', 'original_text', 'lang', 'retweet_count', 'original_author',
       'sentiment_class'],
      dtype='object')

In [0]:
from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', random_state=0)
svm.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
svm_predict = svm.predict(X_test)

In [0]:
#Confusion matrix SVM
from sklearn.metrics import confusion_matrix
svm = confusion_matrix(y_test, svm_predict)
svm

array([[  0, 230,   0],
       [  0, 488,   0],
       [  0, 252,   1]])

In [0]:
from sklearn.metrics import classification_report
svm_score =classification_report(y_test, svm_predict)
print(svm_score)

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       230
           0       0.50      1.00      0.67       488
           1       1.00      0.00      0.01       253

    accuracy                           0.50       971
   macro avg       0.50      0.33      0.23       971
weighted avg       0.51      0.50      0.34       971



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#model building RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=50, criterion='entropy', random_state=0)
classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
 #predicting tweets
 ran_predict = classifier.predict(X_test)

In [0]:
#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, ran_predict)
cm

array([[  5, 223,   2],
       [ 15, 462,  11],
       [  2, 243,   8]])

In [0]:
from sklearn.metrics import classification_report
cr = classification_report(y_test,ran_predict, digits=3)
print(cr)

              precision    recall  f1-score   support

          -1      0.227     0.022     0.040       230
           0      0.498     0.947     0.653       488
           1      0.381     0.032     0.058       253

    accuracy                          0.489       971
   macro avg      0.369     0.333     0.250       971
weighted avg      0.403     0.489     0.353       971



In [0]:
#Naive bayes classifier
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB()
nb_classifier.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
#predicting tweets
nb_predict = nb_classifier.predict(X_test)

In [0]:
#confusion matrix
nb_cm = confusion_matrix(y_test,nb_predict)
nb_cm

array([[ 52, 102,  76],
       [ 90, 211, 187],
       [ 39, 107, 107]])

In [0]:
nb_score = classification_report(y_test,nb_predict)
print(nb_score)

              precision    recall  f1-score   support

          -1       0.29      0.23      0.25       230
           0       0.50      0.43      0.46       488
           1       0.29      0.42      0.34       253

    accuracy                           0.38       971
   macro avg       0.36      0.36      0.35       971
weighted avg       0.40      0.38      0.38       971



In [0]:
#model building XGBClassifier
from xgboost import XGBClassifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
#prediction
xgb_predict = xgb_classifier.predict(X_test)

In [0]:
#Confusion matrix XG Boost
xgb_cm = confusion_matrix(y_test, xgb_predict)
xgb_cm

array([[  4, 225,   1],
       [ 10, 475,   3],
       [  1, 248,   4]])

In [0]:
xgb_score = classification_report(y_test,xgb_predict)
print(xgb_score)

              precision    recall  f1-score   support

          -1       0.27      0.02      0.03       230
           0       0.50      0.97      0.66       488
           1       0.50      0.02      0.03       253

    accuracy                           0.50       971
   macro avg       0.42      0.34      0.24       971
weighted avg       0.45      0.50      0.35       971



In [35]:
#model building Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
gbc= GradientBoostingClassifier()
gbc.fit(X_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
gbc_predict = gbc.predict(X_test)

In [37]:
#Confusion matrix XG Boost
gbc = confusion_matrix(y_test, gbc_predict)
gbc

array([[  5, 225,   0],
       [ 14, 469,   5],
       [  6, 246,   1]])

In [38]:
gbc_score = classification_report(y_test,gbc_predict)
print(gbc_score)

              precision    recall  f1-score   support

          -1       0.20      0.02      0.04       230
           0       0.50      0.96      0.66       488
           1       0.17      0.00      0.01       253

    accuracy                           0.49       971
   macro avg       0.29      0.33      0.23       971
weighted avg       0.34      0.49      0.34       971

